In [104]:
import time
import tracemalloc
from collections import deque

def get_next_states(state, M_total, C_total, capacity):
    M_left, C_left, boat = state
    states = []
    for m in range(capacity + 1):
        for c in range(capacity + 1):
            if m + c == 0 or m + c > capacity:
                continue
            if boat == 0: # If boat is on the left
                new_M_left = M_left - m
                new_C_left = C_left - c
            else:
                new_M_left = M_left + m
                new_C_left = C_left + c

            if 0 <= new_M_left <= M_total and 0 <= new_C_left <= C_total: # Make sure don't exceed total numbers
                M_right = M_total - new_M_left
                C_right = C_total - new_C_left

                if (new_M_left == 0 or new_M_left >= new_C_left) and (M_right == 0 or M_right >= C_right): # Check missionaries are never outnumbered
                    states.append((new_M_left, new_C_left, 1 - boat))
    return states

def bfs(start_state, goal_state, M_total, C_total, capacity):
    queue = deque()
    queue.append((start_state, []))  # (current_state, path)
    visited = set()
    visited.add(start_state)
    nodes_visited = 0

    while queue:
        current_state, path = queue.popleft()
        nodes_visited += 1

        if current_state == goal_state:
            return path + [current_state], nodes_visited

        for next_state in get_next_states(current_state, M_total, C_total, capacity):
            if next_state not in visited:
                visited.add(next_state)
                queue.append((next_state, path + [current_state]))
    return None, nodes_visited

def run_tests():
    tests = [
        (7, 5, 4)
    ]

    for idx, (M, C, cap) in enumerate(tests, 1):
        start = (M, C, 0)
        goal = (0, 0, 1)

        tracemalloc.start()
        start_time = time.perf_counter()

        result, visited_count = bfs(start, goal, M, C, cap)

        end_time = time.perf_counter()
        current, peak = tracemalloc.get_traced_memory()
        tracemalloc.stop()

        if result:
            print(f"\nTest {idx}: M = {M}, C = {C}, capacity = {cap} => Solved in {len(result)-1} moves")
            #print(f"Number of nodes visited: {visited_count}")
            print(f"Excution Time: {end_time - start_time:.6f} seconds")
            print(f"Currently memory used: {current/1024:.2f} KB, Peak: {peak/1024:.2f} KB")
            print_solution(result, M, C)
        else:
            print(f"Test {idx}: M = {M}, C = {C}, capacity = {cap} => No solution")
            #print(f"Number of nodes visited: {visited_count}")
            print(f"Excution Time: {end_time - start_time:.6f} seconds")
            print(f"Currently memory used: {current/1024:.2f} KB, Peak: {peak/1024:.2f} KB")

def print_solution(path, M_total = 3, C_Total = 3):
    print("\nSteps to goal:")
    for step_num in range(1, len(path)):
        prev_state = path[step_num - 1]
        curr_state = path[step_num]

        # Each state is (M_left, C_left, boat_pos)
        prev_m, prev_c, prev_boat = prev_state
        curr_m, curr_c, curr_boat = curr_state

        # Calculate move difference
        m_change = prev_m - curr_m
        c_change = prev_c - curr_c

        # If boat moves to right bank
        if prev_boat == 0:
            direction = "to right bank"
            m_carried = m_change
            c_carried = c_change
        else:
            # Boat moves to left bank
            direction = "to left bank"
            m_carried = -m_change
            c_carried = -c_change

        print(f"Step {step_num}: Boat carries {m_carried}M and {c_carried}C {direction}")
        print(f"  Left bank: {curr_m}M, {curr_c}C | Right bank: {M_total - curr_m}M, {C_Total - curr_c}C")
        print("")
        
if __name__ == "__main__":
    run_tests()


Test 1: M = 7, C = 5, capacity = 4 => Solved in 7 moves
Excution Time: 0.001843 seconds
Currently memory used: 0.53 KB, Peak: 4.49 KB

Steps to goal:
Step 1: Boat carries 0M and 3C to right bank
  Left bank: 7M, 2C | Right bank: 0M, 3C

Step 2: Boat carries 0M and 1C to left bank
  Left bank: 7M, 3C | Right bank: 0M, 2C

Step 3: Boat carries 3M and 1C to right bank
  Left bank: 4M, 2C | Right bank: 3M, 3C

Step 4: Boat carries 0M and 1C to left bank
  Left bank: 4M, 3C | Right bank: 3M, 2C

Step 5: Boat carries 2M and 2C to right bank
  Left bank: 2M, 1C | Right bank: 5M, 4C

Step 6: Boat carries 0M and 1C to left bank
  Left bank: 2M, 2C | Right bank: 5M, 3C

Step 7: Boat carries 2M and 2C to right bank
  Left bank: 0M, 0C | Right bank: 7M, 5C

